# Libraries

In [2]:
import numpy as np
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# SQLite Database Creation

In [61]:
file_list = ['LFB Incident data from January 2009 to December 2012.xlsx',
             'LFB Incident data from January 2013 to December 2016.xlsx',
             'LFB Incident data from January 2017.xlsx',
             'LFB Incident data January 2019 to August 2022.xslx']

mobil_list = ['LFB Mobilisation data from January 2009.csv',
              'LFB Mobilisation data from January 2009 to 2015.csv',
              'LFB Mobilisation data from January 2016.csv',
              'LFB Mobilisation data Last 3 years January 2019 to August 2022.xlsx']

def create_df(file_name):
    path = '/mnt/data/public/lfb/'
    
    columns = ['IncidentNumber',
               'DateOfCall',
               'CalYear',
               'HourOfCall',
               'IncidentGroup',
               'StopCodeDescription',
               'PropertyCategory',
               'IncGeo_BoroughName',
               'IncGeo_WardNameNew',
               'IncidentStationGround']

    return pd.read_excel(f'{path}{file_name}',
                         usecols=columns,
                         engine='openpyxl')

def create_mobil(file_name):
    path = '/mnt/data/public/lfb/mobilisation/'
    
    columns = ['IncidentNumber',
               'CalYear',
               'HourOfCall',
               'ResourceMobilisationId',
               'Resource_Code',
               'PerformanceReporting',
               'TurnoutTimeSeconds',
               'TravelTimeSeconds',
               'AttendanceTimeSeconds',
               'DeployedFromStation_Code',
               'DeployedFromStation_Name',
               'PumpOrder']
    try:
        if file_name[-3:] == 'csv':
            return pd.read_csv(f'{path}{file_name}',
                               usecols=columns,
                               low_memory=False)
        elif file_name[-4:] == 'xlsx':
            return pd.read_excel(f'{path}{file_name}',
                                 usecols=columns,
                                 engine='openpyxl')
    except UnicodeDecodeError:
        pass

def concat_df(*args):
    df = pd.DataFrame()
    for df_file in args:
        df = pd.concat([df, df_file], sort=False)
    df = df.drop_duplicates(ignore_index=True)
#     df = df.dropna()
    return df

def merge_df(df1, df2):
    df = df1.merge(df2,
                   how='inner',
                   on='IncidentNumber')
    df = df.sort_values(['DateOfCall', 'TimeOfCall']).reset_index(drop=True)
    return df

def conv_date_str(df_lfb):
    df_lfb['DateOfCall'] = df_lfb['DateOfCall'].astype('str')
    return None

def create_db(database):
    conn = sqlite3.connect(f'/mnt/processed/private/msds2023/lt9/{database}')
    df_lfb.to_sql(name='df_lfb', con=conn, if_exists='replace', index=False)
    df_mob.to_sql(name='df_mob', con=conn, if_exists='replace', index=False)
    conn.close()
    return None

def treat_nulls(df):
    category_list = df['CalYear'].unique().tolist()
    for j in category_list:
        mean = df.loc[df['CalYear'] == j]['TurnoutTimeSeconds'].mean(skipna=True)
        df.loc[(df['CalYear'] == j) &
               (df['TurnoutTimeSeconds'].isna()), 'TurnoutTimeSeconds'] = mean
        
    df.loc[df['TravelTimeSeconds'].isna(), 'TravelTimeSeconds'] = df['AttendanceTimeSeconds'] - df['TurnoutTimeSeconds']
    return None

def rem_null(df):
    df = df.dropna()
    return df

def rem_negative(df):
    df_mob.loc[df_mob['TravelTimeSeconds'] < 0, 'TravelTimeSeconds'] = np.nan
    return None

## Incident Database

In [4]:
df_incident_2012 = create_df(file_list[0])
df_incident_2016 = create_df(file_list[1])
df_incident_2017 = create_df(file_list[2])
df_incident_2022 = create_df(file_list[3])

df_lfb = concat_df(df_incident_2012,
                   df_incident_2016,
                   df_incident_2017,
                   df_incident_2022)
df_lfb

,IncidentNumber,DateOfCall,CalYear,HourOfCall,IncidentGroup,StopCodeDescription,PropertyCategory,IncGeo_BoroughName,IncGeo_WardNameNew,IncidentStationGround
0,235138081,2009-01-01 00:00:00,2009,0,Special Service,Special Service,Road Vehicle,WANDSWORTH,Queenstown,Battersea
1,1091,2009-01-01 00:00:00,2009,0,Special Service,Special Service,Outdoor,LAMBETH,Bishop's,Lambeth
2,2091,2009-01-01 00:00:00,2009,0,Fire,Secondary Fire,Outdoor,ENFIELD,Haselbury,Edmonton
3,3091,2009-01-01 00:00:00,2009,0,Fire,Secondary Fire,Outdoor,HILLINGDON,Hillingdon East,Hillingdon
4,5091,2009-01-01 00:00:00,2009,0,Fire,Secondary Fire,Outdoor,ISLINGTON,Holloway,Holloway
...,...,...,...,...,...,...,...,...,...,...
1733121,145831-31082022,31 Aug 2022,2022,23,Fire,Primary Fire,Dwelling,SOUTHWARK,CAMBERWELL GREEN,Old Kent Road
1733122,145832-31082022,31 Aug 2022,2022,23,False Alarm,AFA,Dwelling,CAMDEN,HOLBORN & COVENT GARDEN,Soho
1733123,145834-31082022,31 Aug 2022,2022,23,Fire,Secondary Fire,Outdoor,BRENT,BARNHILL,Wembley
1733124,145839-31082022,31 Aug 2022,2022,23,Special Service,Special Service,Outdoor,HACKNEY,HACKNEY DOWNS,Stoke Newington


In [44]:
conv_date_str(df_lfb)

In [50]:
df_lfb = rem_null(df_lfb)

## Mobilisation Database

In [7]:
df_mobil_2012 = create_mobil(mobil_list[0])
df_mobil_2015 = create_mobil(mobil_list[1])
df_mobil_2016 = create_mobil(mobil_list[2])
df_mobil_2022 = create_mobil(mobil_list[3])

df_mob = concat_df(df_mobil_2012,
                   df_mobil_2015,
                   df_mobil_2016,
                   df_mobil_2022)
df_mob

,IncidentNumber,CalYear,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,DeployedFromStation_Code,DeployedFromStation_Name,PumpOrder
0,235138081,2009,0,38426,H271,1,NaN,NaN,319,H27,Battersea,1
1,235138081,2009,0,38427,H212,2,253.0,89.0,342,H21,Clapham,2
2,2091,2009,0,38429,A341,1,151.0,157.0,308,A34,Edmonton,1
3,3091,2009,0,38430,G232,1,108.0,102.0,210,G23,Hillingdon,1
4,5091,2009,0,38432,A311,1,114.0,119.0,233,A31,Holloway,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2062844,060575-22052021,2021,15,5846050,H382,2,43.0,262.0,305,H38,Sutton,2
2062845,080897-29062021,2021,9,5870157,E381,2,79.0,524.0,603,E38,New Cross,2
2062846,080897-29062021,2021,9,5870165,E351,1,NaN,NaN,1,E37,Peckham,1
2062847,080918-29062021,2021,10,5870193,F331,1,45.0,323.0,368,F33,Whitechapel,1


In [46]:
treat_nulls(df_mob)

In [54]:
rem_negative(df_mob)

In [51]:
df_mob = rem_null(df_mob)

## SQLite Database and Tables

In [59]:
create_db('lfb.db')

In [62]:
!ls /mnt/processed/private/msds2023/lt9

lfb.db


In [1]:
!cp /mnt/processed/private/msds2023/lt9/fide.db ~